# Imports and Setup
To begin, run the cell(s) below to install the necessary setup and imports.

In [1]:
!sudo apt install python3.8

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  libpython3.8-minimal libpython3.8-stdlib python3.8-minimal
Suggested packages:
  python3.8-venv binfmt-support
The following NEW packages will be installed:
  libpython3.8-minimal libpython3.8-stdlib python3.8 python3.8-minimal
0 upgraded, 4 newly installed, 0 to remove and 20 not upgraded.
Need to get 4,695 kB of archives.
After this operation, 18.5 MB of additional disk space will be used.
Get:1 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic/main amd64 libpython3.8-minimal amd64 3.8.14-1+bionic1 [762 kB]
Get:2 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic/main amd64 python3.8-minimal amd64 3.8.14-1+bionic1 [1,839 kB]
Get:3 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic/main am

In [ ]:
!python3.8 --version

Python 3.8.14


In [2]:
%%bash
MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

PREFIX=/usr/local
installing: python-3.6.5-hc3d631a_2 ...
installing: ca-certificates-2018.03.07-0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: libgcc-ng-7.2.0-hdf63c60_3 ...
installing: libstdcxx-ng-7.2.0-hdf63c60_3 ...
installing: libffi-3.2.1-hd88cf55_4 ...
installing: ncurses-6.1-hf484d3e_0 ...
installing: openssl-1.0.2o-h20670df_0 ...
installing: tk-8.6.7-hc745277_3 ...
installing: xz-5.2.4-h14c3975_4 ...
installing: yaml-0.1.7-had09818_2 ...
installing: zlib-1.2.11-ha838bed_2 ...
installing: libedit-3.1.20170329-h6b74fdf_2 ...
installing: readline-7.0-ha6073c6_4 ...
installing: sqlite-3.23.1-he433501_0 ...
installing: asn1crypto-0.24.0-py36_0 ...
installing: certifi-2018.4.16-py36_0 ...
installing: chardet-3.0.4-py36h0f667ec_1 ...
installing: idna-2.6-py36h82fb2a8_1 ...
installing: pycosat-0.6.3-py36h0a5515d_0 ...
installing: pycparser-2.18-py36hf9f622e_1 ...
installing: pysocks-1.6.8-py36_0 ...
installing: ruamel_yaml-0.15.37-py36h14c3975_2 ...
installing: six-1.11

--2022-09-25 14:34:44--  https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79, 2606:4700::6812:c94f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh [following]
--2022-09-25 14:34:45--  https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58468498 (56M) [application/x-sh]
Saving to: ‘Miniconda3-4.5.4-Linux-x86_64.sh’

     0K .......... .......... .......... .......... ..........  0% 62.0M 1s
    50K .......... .......... .......... .......... ..........  0%

In [3]:
%%bash
conda install --channel defaults conda python=3.8 --yes
conda update --channel defaults --all --yes

Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs: 
    - conda
    - python=3.8


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    idna-3.3                   |     pyhd3eb1b0_0          55 KB
    ld_impl_linux-64-2.38      |       h1181459_1         732 KB
    setuptools-63.4.1          |   py38h06a4308_0         1.3 MB
    wheel-0.37.1               |     pyhd3eb1b0_0          31 KB
    conda-4.14.0               |   py38h06a4308_0        1009 KB
    cffi-1.15.1                |   py38h74dc2b5_0         230 KB
    cytoolz-0.11.0             |   py38h7b6447c_0         390 KB
    pysocks-1.7.1              |   py38h06a4308_0          31 KB
    pycosat-0.6.3              |   py38h7b6447c_1         107 KB
    yaml-0.2.5                 |       h7b6447c_0          87 KB
    readline-8.1.2             |       h7f8727e_1   

idna-3.3             |   55 KB | ########## | 100% 
ld_impl_linux-64-2.3 |  732 KB | ########## | 100% 
setuptools-63.4.1    |  1.3 MB | ########## | 100% 
wheel-0.37.1         |   31 KB | ########## | 100% 
conda-4.14.0         | 1009 KB | ########## | 100% 
cffi-1.15.1          |  230 KB | ########## | 100% 
cytoolz-0.11.0       |  390 KB | ########## | 100% 
pysocks-1.7.1        |   31 KB | ########## | 100% 
pycosat-0.6.3        |  107 KB | ########## | 100% 
yaml-0.2.5           |   87 KB | ########## | 100% 
readline-8.1.2       |  423 KB | ########## | 100% 
sqlite-3.39.2        |  1.5 MB | ########## | 100% 
certifi-2022.9.14    |  158 KB | ########## | 100% 
tk-8.6.12            |  3.3 MB | ########## | 100% 
zlib-1.2.12          |  124 KB | ########## | 100% 
python-3.8.13        | 22.7 MB | ########## | 100% 
ncurses-6.3          |  1.1 MB | ########## | 100% 
charset-normalizer-2 |   33 KB | ########## | 100% 
brotlipy-0.7.0       |  349 KB | ########## | 100% 
ruamel_yaml-

In [4]:
!conda update -n base -c defaults conda --yes

Solving environment: \ | / - \ | / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::chardet==3.0.4=py36h0f667ec_1
  - defaults/linux-64::six==1.11.0=py36h372c433_1
  - defaults/linux-64::asn1crypto==0.24.0=py36_0
- \ done

# All requested packages already installed.

Retrieving notices: ...working... done


In [5]:
import sys
_ = (sys.path
        .append("/usr/local/lib/python3.8/site-packages"))

In [6]:
import sys
sys.path

['/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/usr/local/lib/python3.8/site-packages']

In [7]:
!conda init zsh

no change     /usr/local/condabin/conda
no change     /usr/local/bin/conda
no change     /usr/local/bin/conda-env
no change     /usr/local/bin/activate
no change     /usr/local/bin/deactivate
no change     /usr/local/etc/profile.d/conda.sh
no change     /usr/local/etc/fish/conf.d/conda.fish
no change     /usr/local/shell/condabin/Conda.psm1
no change     /usr/local/shell/condabin/conda-hook.ps1
no change     /usr/local/lib/python3.8/site-packages/xontrib/conda.xsh
no change     /usr/local/etc/profile.d/conda.csh
modified      /root/.zshrc

==> For changes to take effect, close and re-open your current shell. <==



In [8]:
!conda create --name env python=3.8 --yes

Solving environment: \ done

## Package Plan ##

  environment location: /usr/local/envs/env

  added / updated specs:
    - python=3.8


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2022.07.19 |       h06a4308_0         124 KB
    certifi-2022.9.14          |   py38h06a4308_0         154 KB
    ld_impl_linux-64-2.38      |       h1181459_1         654 KB
    libffi-3.3                 |       he6710b0_2          50 KB
    libgcc-ng-11.2.0           |       h1234567_1         5.3 MB
    libstdcxx-ng-11.2.0        |       h1234567_1         4.7 MB
    ncurses-6.3                |       h5eee18b_3         781 KB
    openssl-1.1.1q             |       h7f8727e_0         2.5 MB
    pip-22.1.2                 |   py38h06a4308_0         2.5 MB
    python-3.8.13              |       h12debd9_0        18.8 MB
    readline-8.1.2             |       h7f8727e_1         354 KB
 

In [9]:
!source /usr/local/bin/activate env

In [10]:
%%bash
pip install torch==1.10.1+cu111 torchvision==0.11.2+cu111 torchaudio==0.10.1 -f https://download.pytorch.org/whl/torch_stable.html
pip install timm==0.4.12
pip install pickle5
pip install scikit-learn
pip install numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 GB 806.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 70.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.0/377.0 kB 18.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py)

tcmalloc: large alloc 2137669632 bytes == 0x561a661ba000 @  0x7f9314fa01e7 0x561a63f71593 0x561a63f71620 0x561a63fc3bcd 0x561a63fc4988 0x561a63fd1d15 0x561a6402be60 0x561a64021600 0x561a64022b64 0x561a6402be60 0x561a64021600 0x561a64022b64 0x561a6402be60 0x561a64021600 0x561a64022b64 0x561a6402be60 0x561a64021600 0x561a64022b64 0x561a6402be60 0x561a64021600 0x561a64022b64 0x561a6402be60 0x561a640227e7 0x561a6402be60 0x561a64021600 0x561a64022bc4 0x561a6402ceb0 0x561a64021600 0x561a64022b64 0x561a6402be60 0x561a64021600
tcmalloc: large alloc 2672091136 bytes == 0x561ae585e000 @  0x7f9314fa1615 0x561a63fc3bcd 0x561a63fc4988 0x561a63fd1d15 0x561a6402be60 0x561a64021600 0x561a64022b64 0x561a6402be60 0x561a64021600 0x561a64022b64 0x561a6402be60 0x561a64021600 0x561a64022b64 0x561a6402be60 0x561a64021600 0x561a64022b64 0x561a6402be60 0x561a64021600 0x561a64022b64 0x561a6402be60 0x561a640227e7 0x561a6402be60 0x561a64021600 0x561a64022bc4 0x561a6402ceb0 0x561a64021600 0x561a64022b64 0x561a6402

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
!pwd

/content


In [13]:
%cd /content/drive/MyDrive/Toyota_Smarthome-main/pipline/

/content/drive/.shortcut-targets-by-id/1LWR6jlRpEpJnHZO4yBjJ5EPNy1QfYCYq/Toyota_Smarthome-main/pipline


In [ ]:
!python3.8 /content/drive/MyDrive/Toyota_Smarthome-main/pipline/train.py --help

usage: train.py
       [-h]
       [-mode MODE]
       [-train TRAIN]
       [-comp_info COMP_INFO]
       [-rgb_model_file RGB_MODEL_FILE]
       [-flow_model_file FLOW_MODEL_FILE]
       [-gpu GPU]
       [-dataset DATASET]
       [-rgb_root RGB_ROOT]
       [-flow_root FLOW_ROOT]
       [-type TYPE]
       [-lr LR]
       [-epoch EPOCH]
       [-model MODEL]
       [-APtype APTYPE]
       [-randomseed RANDOMSEED]
       [-load_model LOAD_MODEL]
       [-num_channel NUM_CHANNEL]
       [-batch_size BATCH_SIZE]
       [-kernelsize KERNELSIZE]
       [-feat FEAT]
       [-split_setting SPLIT_SETTING]

optional arguments:
  -h, --help
    show this
    help
    message and
    exit
  -mode MODE
    rgb or flow
    (or joint
    for eval)
  -train TRAIN
    train or
    eval
  -comp_info COMP_INFO
  -rgb_model_file RGB_MODEL_FILE
  -flow_model_file FLOW_MODEL_FILE
  -gpu GPU
  -dataset DATASET
  -rgb_root RGB_ROOT
  -flow_root FLOW_ROOT
  -type TYPE
  -lr LR
  -epoch EPOCH
  -model MODEL

In [ ]:
!python3.8 /content/drive/MyDrive/Toyota_Smarthome-main/pipline/train.py \
-dataset TSU \
-mode rgb \
-split_setting CS \
-model PDAN \
-num_channel 512 \
-lr 0.0002 \
-kernelsize 3 \
-APtype map \
-epoch 15 \
-batch_size 1 \
-comp_info TSU_CS_RGB_PDAN \
-load_model /content/drive/MyDrive/Toyota_Smarthome-main/model/PDAN_TSU_RGB \
-rgb_model_file P02T01C06

Random_SEED!!!: 0
PDAN
batch_size: 1
cuda_avail True
RGB mode ../feature_file/RGB_i3d_16frames_64000_SSD
Traceback (most recent call last):
  File "/content/drive/MyDrive/Toyota_Smarthome-main/pipline/train.py", line 295, in <module>
    dataloaders, datasets = load_data(train_split, test_split, rgb_root)
  File "/content/drive/MyDrive/Toyota_Smarthome-main/pipline/train.py", line 129, in load_data
    dataset = Dataset(train_split, 'training', root, batch_size, classes)
  File "/content/drive/.shortcut-targets-by-id/1LWR6jlRpEpJnHZO4yBjJ5EPNy1QfYCYq/Toyota_Smarthome-main/pipline/smarthome_i3d_per_video.py", line 59, in __init__
    self.data = make_dataset(split_file, split, root, classes)
  File "/content/drive/.shortcut-targets-by-id/1LWR6jlRpEpJnHZO4yBjJ5EPNy1QfYCYq/Toyota_Smarthome-main/pipline/smarthome_i3d_per_video.py", line 30, in make_dataset
    data = json.load(f)
  File "/usr/local/lib/python3.8/json/__init__.py", line 293, in load
    return loads(fp.read(),
  File "/usr/

In [14]:
!pip install ipywidgets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.4/134.4 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 74.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.1/384.1 kB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 752.0/752.0 kB 63.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.1/107.1 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 73.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.0/424.0 kB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.8/284.8 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 76.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━

## Initializing Constants

In [20]:
# INITIALIZING PATH VARIABLES (CHANGE HERE IF DIFFERENT DIRECTORY)

PATH_TO_MODEL_FOLDER = "/content/drive/MyDrive/Toyota_Smarthome-main/model/"
PATH_TO_VIDEO_FOLDER = "/content/drive/MyDrive/Toyota_Smarthome-main/video/"
PATH_TO_FEATURE_FOLDER = "/content/drive/MyDrive/Toyota_Smarthome-main/feature_file/"

# R2

Data Exploration Section:
- Check out the different input videos (with UI)
- See playback of video

In [21]:
import os
import ipywidgets as widgets

videoList = os.listdir(PATH_TO_VIDEO_FOLDER)

video = widgets.Dropdown(
    options = videoList,
    description='Video:',
    disabled=False,
)
video

Dropdown(description='Video:', options=('sample_video_2.mp4', 'sample_video_1.mp4'), value='sample_video_2.mp4…

In [22]:
from IPython.display import Video

VIDEO_SELECTED = PATH_TO_VIDEO_FOLDER + video.value
Video(VIDEO_SELECTED)

# R3
Inference Section:
- Load a pretrained model (with UI)
- Choose input video (with UI)
- See inference result in the form out output video with captions that indicte the current detected activity in each video frame

In [ ]:
import os
import ipywidgets as widgets

modelList = os.listdir(PATH_TO_MODEL_FOLDER)

for i in modelList:
  if "." in i:  # i.find(".") == 1
    modelList.remove(i)

model = widgets.Dropdown(
    options = modelList,
    description='Model:',
    disabled=False,
)

box = widgets.Box(children = [model, video])
box

Box(children=(Dropdown(description='Model:', options=('PDAN_TSU_RGB', '.ipynb_checkpoints'), value='PDAN_TSU_R…

In [ ]:
#finalized chosen model and video directory

SELECTED_MODEL = PATH_TO_MODEL_FOLDER + model.value
SELECTED_VIDEO = PATH_TO_VIDEO_FOLDER + video.value
print(SELECTED_MODEL)
print(SELECTED_VIDEO)

/content/drive/MyDrive/Colab Notebooks/ICT3104/Toyota_Smarthome-main/model/PDAN_TSU_RGB
/content/drive/MyDrive/Colab Notebooks/ICT3104/Toyota_Smarthome-main/video/sample_video_2.mp4


# FEATURE EXTRACTION

TSU I3D Feature Extraction from video

In [ ]:
%cd /content/drive/MyDrive/Toyota_Smarthome-main/feature_extract/

! git clone https://github.com/v-iashin/video_features.git
! pip install omegaconf==2.0.6 av==8.0.2

/content/drive/.shortcut-targets-by-id/1LWR6jlRpEpJnHZO4yBjJ5EPNy1QfYCYq/Toyota_Smarthome-main/feature_extract
Cloning into 'video_features'...
remote: Enumerating objects: 1283, done.
remote: Counting objects: 100% (384/384), done.
remote: Compressing objects: 100% (165/165), done.
remote: Total 1283 (delta 248), reused 298 (delta 203), pack-reused 899
Receiving objects: 100% (1283/1283), 288.54 MiB | 17.11 MiB/s, done.
Resolving deltas: 100% (664/664), done.
Checking out files: 100% (177/177), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pwd

/content/drive/.shortcut-targets-by-id/1LWR6jlRpEpJnHZO4yBjJ5EPNy1QfYCYq/Toyota_Smarthome-main/feature_extract


In [ ]:
%cd /content/drive/MyDrive/Toyota_Smarthome-main/feature_extract/video_features

/content/drive/.shortcut-targets-by-id/1LWR6jlRpEpJnHZO4yBjJ5EPNy1QfYCYq/Toyota_Smarthome-main/feature_extract/video_features


In [ ]:
from models.i3d.extract_i3d import ExtractI3D
from utils.utils import build_cfg_path
from omegaconf import OmegaConf
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.cuda.get_device_name(0)

# TODO: RE-ADJUST THE DIRECTORY TO BE TO FEATURE_FILE

# TODO: RE-ADJUST TRAIN_CUSTOM.PY

In [ ]:
# Select the feature type
feature_type = 'i3d'

# Load and patch the config
args = OmegaConf.load(build_cfg_path(feature_type))
args.video_paths = ['/content/drive/MyDrive/Toyota_Smarthome-main/pipline/data/videos/sample_videos/sample_video_1.mp4']
# args.show_pred = True
# args.stack_size = 24
# args.step_size = 24
# args.extraction_fps = 25
args.flow_type = 'raft' # 'pwc' is not supported on Google Colab (cupy version mismatch)
# args.streams = 'flow'

# Load the model
extractor = ExtractI3D(args)

# Extract features
for video_path in args.video_paths:
    # extract name of video from video path (and remove '.mp4' suffix)
    video_name = (video_path.split('/')[-1]).split('.')[0]
    print(video_name)
    print(f'Extracting for {video_path}')
    feature_dict = extractor.extract(video_path)
    print(feature_dict.items())
    for k, v in feature_dict.items():
      print(k)
      print(v.shape)
      print(v)
      if k == "rgb":
        np.save(f'/content/drive/MyDrive/Toyota_Smarthome-main/pipline/data/extracted_features/rgb/{video_name}_rgb', v)
      elif k == "flow":
        np.save(f'/content/drive/MyDrive/Toyota_Smarthome-main/pipline/data/extracted_features/flow/{video_name}_flow', v)
      elif k == "fps":
        np.save(f'/content/drive/MyDrive/Toyota_Smarthome-main/pipline/data/extracted_features/fps/{video_name}_fps', v)
      elif k == "timestamps_ms":
        np.save(f'/content/drive/MyDrive/Toyota_Smarthome-main/pipline/data/extracted_features/timestamps_ms/{video_name}_timestamps_ms', v)
      else:
        raise NotImplementedError("Error: Unexpected item in feature_dict!")

In [ ]:
import numpy as np
data = np.load('/content/drive/MyDrive/Toyota_Smarthome-main/pipline/data/extracted_features/rgb/sample_video_1_rgb.npy')
print(data)

# R4 

- Choose a dataset subfolder to use for training (with UI)
- Initialize a model with a network architecture configured in a separate .py file

In [25]:
import os
import ipywidgets as widgets

featureFolder = os.listdir(PATH_TO_FEATURE_FOLDER)

feature = widgets.Dropdown(
    options = featureFolder,
    description='Feature:',
    disabled=False,
)
feature


Dropdown(description='Feature:', options=('RGB_i3d_16frames_64000_SSD',), value='RGB_i3d_16frames_64000_SSD')

In [26]:
SELECTED_FEATURE = PATH_TO_FEATURE_FOLDER + feature.value
print(SELECTED_FEATURE)

/content/drive/MyDrive/Toyota_Smarthome-main/feature_file/RGB_i3d_16frames_64000_SSD
